In [ ]:
%run /home/ec2-user/trading/Data/cloud_database_connector.ipynb
import requests
from kiteconnect import KiteConnect
import logging
import datetime
import numpy as np
from pypfopt import HRPOpt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
import pytz
import json
from datetime import datetime
# getting utc timezone
utc = pytz.utc
# getting timezone by name
ist = pytz.timezone('Asia/Kolkata')
# getting datetime of specified timezone
now = datetime.now(tz=ist)
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
#### Connect to Data Feed #######
##################
kite = KiteConnect(api_key = "key") 
logging.basicConfig(level=logging.DEBUG)
kite.login_url()

In [ ]:
data = kite.generate_session("key", api_secret="key")
kite.set_access_token(data["access_token"])
token = data["access_token"]


In [ ]:
############ Fetch Symbols and Respective Tokens From Zerodha ###############
instrument_dump = kite.instruments("NSE")
def instrumentLookup(symbol):
    instrument_df = pd.DataFrame(instrument_dump)
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
j = requests.get(url="https://masterswift.mastertrust.co.in/api/v2/contracts.json?exchanges=NSE")

# looks to be json so let's see if json will load it
content = json.loads(j.content)
mt_df = pd.DataFrame.from_dict(content['NSE'])


df = pd.read_csv('nifty50.csv')
symbolist = []
for sym in df.Symbol:
    symbolist.append(sym)
    
symbolist = sorted(symbolist)
token_list = [] 
for symbol in symbolist:
    token_list.append(instrumentLookup(symbol))
    
token_map = {}
for tok,sym in zip(token_list, symbolist):
    token_map.update({sym.upper() + '-EQ':tok})
    

In [ ]:
##################################### Form A Symbolist and Tokenlist #################################################################
import datetime
dt = datetime.datetime.now()
month = dt.strftime("%B")
def get_trading_symbols():
    futureslist = []
    for tick in tickdatalist:
        if "21APRFUT" in tick:
            futureslist.append(tick) ### get the front month symbols
            
    ###### Get the your securities universe #####
    df = pd.read_csv('/home/ec2-user/trading/Trade/nifty50.csv')
    cashlist  = []
    for sym in df.Symbol:
        cashlist.append(sym)
    #print(cashlist)
    equity_fut =  []
    for cash in cashlist:
        fut_sym = cash + "21APRFUT"
        equity_fut.append(fut_sym)
        
    #print(equity_fut)
    
    currency_fut = ['USDINR21APRFUT', 'EURINR21APRFUT','GBPINR21APRFUT','JPYINR21APRFUT']
    com_list= ['GOLD', 'CRUDE', "NATURALGAS","SILVER"]
    
    
    return cashlist #+ equity_fut
        
    

In [ ]:
import numpy as np
def format_response_filter1(predict_string):
    print("Entering Format_response")
    format_1 = predict_string.rstrip("\n")
    format_2 = format_1.replace(' ','')
    format_3 = format_2.replace('+0.j','')
    format_4  = format_3.replace('{"prediction":"[',"")
    format_5  = format_4.replace(']"}','')
    format_6 = str(format_5)
    format_6 = format_6.replace("\\n","")

    
    return format_6
    
def format_response_filter2(predict_string):
    print("Entering Format_response")
    format_1 = predict_string.replace("\n","")
    format_2 = format_1.replace(' ','')
    format_3 = format_2.replace('+0.j','')
    format_4  = format_3.replace('{"prediction":"[',"")
    format_5  = format_4.replace(']"}','')
    format_5 = format_5.replace('\n','')
    print("Printing format 5", format_5)
    format_6 = str(format_5)
    format_6 = predict_string.rstrip("\n")
    fin = np.fromstring(format_6, sep = ",")
    print("final", fin)
    
    return fin
    

In [ ]:
def sig_dict(preds):
    ltpdict=kite.ltp(token_list) #fetching end of minute price
    latest = [] #iniatiing list
    for token in token_list:  
        latest.append(ltpdict[str(token)]['last_price'])
        
    lat = np.asarray(latest)
    indicator_list = preds > lat
    print(indicator_list)
    signal_dict = {}
    for sym,ind in zip(symbolist,indicator_list):
        signal_dict.update({sym:ind})
    
    trade_list = []
    for key in signal_dict.keys():
        if signal_dict[key] == True:
            trade_list.append(key)
    
    print(trade_list)
    trade_string = ""
    for sym in trade_list:
        trade_string += sym.lower() + " "
        

    
    return trade_string
    
    
    

In [ ]:
################################# Fetch the Symbols to Trade in and Direction  ################################################
### For Now Long Only
signal_dict = {}
def tradelist(symbols, token_list):
    ####### generate string of symbols to feed to endpoint
    
    sym_string = ""
    for sym in symbols:
        sym_string += sym.lower() + "%20"
    sym_string += "?rank=1" ## specific rank here of dmd here, -1 = full rank
    url_string_optdmd = "http://localhost:5000/fit_dmd/" + sym_string
  
    
    ####### Stream to Endpoint to Fit Model###########
    r = requests.post(url_string_optdmd)
    print("Need the array dim to confirm it is working",r.text)
    
    ######## Fetch Latest State of System #############
    ltpdict=kite.ltp(token_list) #fetching end of minute price
    latest = [] #iniatiing list
    for token in token_list:  
        latest.append(ltpdict[str(token)]['last_price']) 
        
    ############# Pull for predictions #################
    latest_string = ""
    for late in latest:
        latest_string += str(late) + "%20"
    size = len(latest_string)
    latest_string  = latest_string[:size - 3]
    url_string_predict = "http://localhost:5000/predict_dmd/" + latest_string
    print(url_string_predict)
    p = requests.post(url_string_predict)
    print(p.text)
    resp = list(p.text.split(","))
    for i in range(len(resp)):
        resp[i] = format_response_filter1(resp[i])
        resp[i] = float(resp[i])
    final = np.asarray(resp)
    #final = format_response_filter1(p.text)
    #final = format_response_filter2(final_1)
    
    ############ Convert to predictions to a signal dictionary ###########
    trade_string = sig_dict(final) 
    #print("Need the array dim to confirm it is working",p.text)
    print("Trade String", trade_string)    
    return(trade_string) ###### return symbols to trade in string format 
    

In [ ]:
pred = tradelist(symbolist, token_list)

In [ ]:
######## Heirarchal Risk Parity ########
from pypfopt import HRPOpt
def get_opt_portfolio_hrp(trade_string):
    df = fit(trade_string, rank = 1)
    ret = df.pct_change()
    hrp = HRPOpt(ret)
    weights = hrp.optimize()
    hrp.portfolio_performance(verbose=True)
    #plotting.plot_dendrogram(hrp)
    
    return(weights)
    

In [ ]:
weight_dict = get_opt_portfolio_hrp(pred)

In [ ]:
#### Mean Variance Optimized ######
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

def get_opt_portfolio_mean_var(df):
    mu = mean_historical_return(df)
    S = CovarianceShrinkage(df).ledoit_wolf()
    
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    return(weights)
    


In [ ]:
##### Form Lists to Pass to Write Order #####

trade_symbolist = []
for sym in weight_dict.keys():
    trade_symbolist.append(sym)

weight_list = []
for sym in weight_dict.keys():
    weight_list.append(weight_dict[sym])

poslist = []
for wei in weight_list:
    poslist.append(0)

trade_token_list = [] 
for sym in trade_symbolist:
    trade_token_list.append(instrumentLookup(sym.upper()))    

ltpdict=kite.ltp(trade_token_list) #fetching end of minute price
latest = [] #iniatiing list

for token in trade_token_list:  
    latest.append(ltpdict[str(token)]['last_price']) 

order_index = 1
        

len(weight_list)
equal_weight_list = []
for i in range(len(weight_list)):
    equal_weight_list.append(1/len(weight_list))

final_weight_list = []
for i,j in zip(weight_list,equal_weight_list):
    final_weight_list.append((i*0.5+j*1.5)/2)

In [ ]:
import pandas as pd
import json
import requests

# download the data
j = requests.get(url="https://masterswift.mastertrust.co.in/api/v2/contracts.json?exchanges=NSE")

# looks to be json so let's see if json will load it
content = json.loads(j.content)
df = pd.DataFrame.from_dict(content['NSE'])

mt_token_map = {}
for sym,tok in zip(mt_df.trading_symbol, mt_df.code):
    mt_token_map.update({sym:tok})



In [ ]:
##### Load capital avaiblabl 
id_list = ['NL0119440']
#capital_dict = {}
#for i in id_list:
    #df_temp = pd.read_sql('SELECT * FROM Investor_14FD050 WHERE Date = "2021-07-16"', engine_cloud_investor_log)
    #capitak.update({sym:tok})

In [ ]:
weight_list = final_weight_list

In [ ]:
import pytz
from datetime import datetime
# getting utc timezone
utc = pytz.utc
# getting timezone by name
ist = pytz.timezone('Asia/Kolkata')
# getting datetime of specified timezone
now = datetime.now(tz=ist)
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
import csv
position_dict = {}
def write_orderr(symbol,perc,price,pos):    
    
    capital = 50000
    posnew=int(capital*perc/price) 
    #print(posnew)
    global order_index
    global current_time
    now = datetime.now(tz=ist)
    current_time = now.strftime("%H:%M:%S")
    position_dict.update({symbol:posnew})
    with open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv', 'a', newline='') as file: 
        writer = csv.writer(file)
        ################

        quantity=posnew-pos
        qty=int(quantity)
        
        if posnew*pos>=0:
            if qty==0:
                pass
            elif qty>0:
                
                writer.writerow([order_index,symbol.upper() + "-EQ",qty,"14FD050","LIMIT",price,current_time])
                order_index = order_index + 1
                
            else:
                writer.writerow([order_index,symbol.upper() + "-EQ",qty,"14FD050","LIMIT",price,current_time])
                order_index = order_index + 1

        elif posnew*pos<0:
            if pos>0:
                writer.writerow([order_index,symbol.upper() + "-EQ",qty,"14FD050","LIMIT",price,current_time])
                order_index = order_index + 1
                
            else: 
                writer.writerow([order_index,symbol.upper() + "-EQ",qty,"14FD050","LIMIT",price,current_time])
                order_index = order_index + 1

In [ ]:
import requests
def request_generator(token,order_side,price,quantity,client_id,auth):
    
    
    url = "https://masterswift-beta.mastertrust.co.in/api/v1/orders?exchange=NSE&instrument_token=" +str(token)+"&order_side=" +str(order_side) + "&order_type=LIMIT&price="+str(price)+"&product=NRML&quantity="+str(quantity)+"&validity=DAY&user_order_id=15&client_id=" +str(client_id)
    
    #url = "https://masterswift-beta.mastertrust.co.in/api/v1/orders?exchange=NSE&instrument_token=" \
    #+ str(token) "&order_side=" # +str(order_side) + "&order_type=LIMIT&price=" # + str(price) + "&product=NRML&quantity=" + "&validity=DAY&user_order_id=\
    payload={}
    headers = {
      'x-device-type': 'WEB',
      'client_id': client_id,
      'x-authorization-token': auth
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)







In [ ]:
############## Get date for today ###########################
from datetime import date
today = date.today()
print(str(today))

In [ ]:
####### Fetch auth tokens for client orders ############
auth_token_dict = {}
id_list =  ['NL0119440']
df = pd.read_csv('/home/ec2-user/trading/Data/Equity Pipeline/Tokens/auth_tokens_' + str(today) + '.csv')
for i,tk in zip(id_list,df.Token):
    auth_token_dict.update({i:tk})

In [ ]:
def get_multiplier(idx):
    account_dict = {'NL0119440' : 1 ,'NL0120070' : 1,'NL0120099' : 1 ,'NL01225' : 1}
    #print(account_dict[idx])
    return account_dict[idx]

In [ ]:
import csv
import random
position_dict = {}

def write_order_multi(token,perc,price,pos):    
    
    capital = 50000
    global order_index
    posnew=int(capital*perc/price) 
    position_dict.update({symbol.upper() + '-EQ':posnew})
    with open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv', 'a', newline='') as file: 
        writer = csv.writer(file)
        ################

        quantity=posnew-pos
        qty=int(quantity)
        
        random.shuffle(id_list)

        for idx in id_list:
            multiplier = get_multiplier(idx)
        
        
            if posnew*pos>=0:
                if qty==0:
                    pass
                elif qty>0:
                    
                    print(token,'BUY',price,qty*get_multiplier(idx),idx,symbol)
                    
                    #request_generator(token,'BUY',price,qty*get_multiplier(idx),idx,auth_token_dict[idx])

                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

                else:
                    request_generator(token,'SELL',price,qty*get_multiplier(idx),idx,auth_token_dict(idx))
                    
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1
                      

            elif posnew*pos<0:
                if pos>0:
                    
                    request_generator(token,'SELL',price,qty*get_multiplier(idx),idx,auth_token_dict(idx))
                    
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

                else: 
                    
                    
                    request_generator(token,'BUY',price,qty*get_multiplier(idx),idx,auth_token_dict(idx))
                    
                    writer.writerow([order_index,symbol.upper() + "-EQ",qty*get_multiplier(idx),idx,"LIMIT",price,current_time])
                    order_index = order_index + 1

In [ ]:
##### Live Server
def store_order():
      
    
    file = "/home/ec2-user/trading/Trade/Positions/orders_stream.csv"
    fil = open(file,"r")
    orders = fil.read()
    fil.close()
    
    wrt = open('/home/ec2-user/trading/Trade/Positions/total_orders' + str(today) + '.csv', 'a') 
    wrt.write(orders)
    wrt.close()
                
    #file = open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv','w+')
    #file.close()
    print("streamed :D")


In [ ]:
###### Re-stream cancelled order ######
import csv
#order_regenerator(order_index,'POWERGRID-EQ',"14FD050",86,219.70)
def order_regenerator(idx, indentifier, client_id, quantity,price):
    with open('/home/ec2-user/trading/Trade/Positions/reorder_stream.csv', 'a', newline='') as file: 
        writer = csv.writer(file)
        writer.writerow([idx,indentifier,quantity,client_id,"LIMIT",price,current_time])
 
    
    

In [ ]:
#### Fetch Latest State and Write and Stream Order ######
ltpdict=kite.ltp(trade_token_list) #fetching end of minute price
latest = [] #iniatiing list

for token in trade_token_list:  
    latest.append(ltpdict[str(token)]['last_price']) 

for symbol,perc,price,pos in zip(trade_symbolist,weight_list,latest,poslist):
    write_order_multi(mt_token_map[symbol.upper() + "-EQ"],perc,price,pos)

store_order()


In [ ]:
idx = order_index

In [ ]:
########### Parse tradefile from broker to CSV #########################

def get_trade_book(client_id, auth_token):
    

    url = "https://masterswift-beta.mastertrust.co.in/api/v1/trades?client_id=" + str(client_id)

    payload={}
    headers = {
      'x-device-type': 'WEB',
      'x-authorization-token': auth_token,
      'client_id': 'REST6'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return json.loads(response.text)

In [ ]:
import json
parameter_list = ['trade_number','trading_symbol','trade_price','trade_quantity', 'client_id', 'trade_time']
outer_list = []

for idx in id_list:
    trade_dict = get_trade_book(idx,auth_token_dict[idx])  

    for i in range(len(trade_dict['data']['trades'])):
        inner_list = []
        for para in parameter_list:
            if para == 'trade_quantity':
                if trade_dict['data']['trades'][i]['order_side'] == 'BUY':
                    qt = trade_dict['data']['trades'][i][para]
                    inner_list.append(qt)
                else:
                    qt = trade_dict['data']['trades'][i][para]
                    inner_list.append(-qt)
                    
                    
            else:
                inner_list.append(trade_dict['data']['trades'][i][para])
        
        outer_list.append(inner_list)

        

        

trade_dataframe = pd.DataFrame(outer_list, columns = ['trade_number','symbol','trade_price','qty', 'client', 'trade_time'])
        
        
        

In [ ]:
import ftplib

#order_index = 1
df = trade_dataframe
random.shuffle(id_list)
for clientid in id_list:
    print(clientid)
    client1_df = df[df.client == clientid]
    client1_dict = {}

    sub_list = list(client1_df.symbol.values)
    for sym,qty in zip(client1_df.symbol,client1_df.qty):
        
        if sub_list.count(sym) == 1:
            client1_dict.update({sym:qty})
        else:
            updated_qty = np.sum(client1_df[client1_df.symbol == sym].qty.values)
            client1_dict.update({sym:updated_qty})
            


    print("Client dict", client1_dict)
    left_position_dict = {}
    
    print(get_multiplier(clientid))
    for key in position_dict.keys():
        try:
            client1_dict[key.upper()]
            
            if client1_dict[key.upper()] != position_dict[key]*get_multiplier(clientid):
                left_position_dict.update({key:position_dict[key]*get_multiplier(clientid)-client1_dict[key.upper()]})
                
               
            
        except:
            if position_dict[key] > 0:
                print("Not existing", key,position_dict[key])
                left_position_dict.update({key:position_dict[key]*get_multiplier(clientid)})#-client1_dict[key.upper()]})
                
            
    print("Left position dict :", left_position_dict)       
    left_token_list = []            
    for key in left_position_dict.keys():
        left_token_list.append(mt_token_map[key.upper()])
    
    ltp_token_list = []
    for key in left_position_dict.keys():
        ltp_token_list.append(token_map[key.upper()])
    
        
    
    latest = [] #iniatiing list
    
    ltpdict=kite.ltp(ltp_token_list)

    for token in ltp_token_list:  
        latest.append(ltpdict[str(token)]['last_price']) 


    for token,sym,lat in zip(left_token_list,left_position_dict.keys(),latest):
        print(token,sym,lat)
        print(token,'BUY',lat,left_position_dict[sym],clientid,auth_token_dict[clientid])
        #request_generator(token,'BUY',lat,left_position_dict[sym],clientid,auth_token_dict[clientid])

        order_regenerator(idx, sym.upper() + "-EQ",clientid,left_position_dict[sym],lat)
        

filepath = "/home/ec2-user/trading/Trade/Positions/reorder_stream.csv"
file = open(filepath,"r")
orders = file.read()
wrt = open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv', 'w+') 
wrt.write(orders)
wrt.close()


filepath = "/home/ec2-user/trading/Trade/Positions/orders_stream.csv"
file = open(filepath,"r")
orders = file.read()
wrt = open('/home/ec2-user/trading/Trade/Positions/resent_orders' + str(today) + '.csv', 'a') 
wrt.write(orders)
wrt.close()
wrt = open('/home/ec2-user/trading/Trade/Positions/reorder_stream.csv', 'w+') 
wrt.close()
store_order()
%time

In [ ]:
########################## Store Existing Positions ###########################################
existing_position = []
for key in position_dict.keys():
    existing_position.append(position_dict[key])

import pickle
a_file = open('/home/ec2-user/trading/Trade/Positions/postion_dict'+ str(today) + '.pkl','wb')
pickle.dump(position_dict, a_file)

In [ ]:
order_index = 1
import pickle
a_file = open('/home/ec2-user/trading/Trade/Positions/postion_dict2021-06-07.pkl','rb') ### Enter the date that needs to be squared off
position_dict = pickle.load(a_file)



In [ ]:
################################ SQUARING OFF #########################
order_index = 1
import pickle
a_file = open('/home/ec2-user/trading/Trade/Positions/postion_dict2021-06-23.pkl','rb') ### Enter the date that needs to be squared off
position_dict = pickle.load(a_file)
####### Clear Existing Positions ##########
existing_position = []
trade_symbolist = []
for key in position_dict.keys():
    existing_position.append(position_dict[key])
    trade_symbolist.append(key.replace("-EQ",""))
weight_list = np.zeros(len(existing_position))


latest = [] #iniatiing list
trade_token_list = [] 

for sym in trade_symbolist:
    trade_token_list.append(instrumentLookup(sym.upper()))  
#print(trade_token_list)
ltpdict=kite.ltp(trade_token_list)    
for token in trade_token_list:
    #print(token)
    latest.append(ltpdict[str(token)]['last_price']) 

for symbol,perc,price,pos in zip(trade_symbolist,weight_list,latest,existing_position):
    write_order_multi(symbol,perc,price,pos)
####### Stream Order #######
stream_order()

In [ ]:
a_file = open('/home/ec2-user/trading/Trade/Positions/postion_dict2021-06-23.pkl','rb') ### Enter the date that needs to be squared off
position_dict = pickle.load(a_file)
for key in position_dict.keys():
    position_dict.update({key:-position_dict[key]})

position_dict

In [ ]:
idx = order_index

In [ ]:
########## For Squaring Off Existing Position Of Earlier Time ###################
import ftplib

# create a new FTP() instance
session = ftplib.FTP()

# connect to our FTP site
session.connect('14.142.196.162' )

# log into the FTP site
session.login('administrator', 'Hello@123!')

####### FINAL #####

filename = 'JagjitTradeFile2.csv'

localfile = open('/home/ec2-user/trading/Trade/Positions/trade_file' + str(today) + '.csv', 'wb')
session.retrbinary('RETR ' + filename, localfile.write, 1024)

session.quit()
localfile.close()

order_index = 1
df = pd.read_csv('/home/ec2-user/trading/Trade/Positions/trade_file' + str(today) + '.csv',names =  ['index','symbol','foo','qty','client','date','time-stamp'])
random.shuffle(id_list)
for clientid in id_list:
    print(clientid)
    client1_df = df[df.client == clientid]
    client1_dict = {}

    sub_list = list(client1_df.symbol.values)
    for sym,qty in zip(client1_df.symbol,client1_df.qty):
        
        if sub_list.count(sym) == 1:
            client1_dict.update({sym:qty})
        else:
            updated_qty = np.sum(client1_df[client1_df.symbol == sym].qty.values)
            client1_dict.update({sym:updated_qty})
            


    print("Client dict", client1_dict)
    left_position_dict = {}
    
    print(get_multiplier(clientid))
    for key in position_dict.keys():
        try:
            client1_dict[key.upper()]
            
            if client1_dict[key.upper()] != position_dict[key]*get_multiplier(clientid):
                left_position_dict.update({key:position_dict[key]*get_multiplier(clientid)-client1_dict[key.upper()]})
                
               
            
        except:
            if position_dict[key] < 0:
                print("Not existing", key,position_dict[key])
                left_position_dict.update({key:position_dict[key]*get_multiplier(clientid)})#-client1_dict[key.upper()]})
                
            
           
    left_token_list = []            
    for key in left_position_dict.keys():
        left_token_list.append(token_map[key.upper()])
        
    print(left_position_dict)
    latest = [] #iniatiing list
    try:
        ltpdict=kite.ltp(left_token_list)

        for token in left_token_list:  
            latest.append(ltpdict[str(token)]['last_price']) 


        for sym,lat in zip(left_position_dict.keys(),latest):
            order_regenerator(idx, sym.upper() + "-EQ",clientid,left_position_dict[sym],lat)
            print(sym,lat)
            idx = idx + 1
    except:
        pass





filepath = "/home/ec2-user/trading/Trade/Positions/reorder_stream.csv"
file = open(filepath,"r")
orders = file.read()
wrt = open('/home/ec2-user/trading/Trade/Positions/orders_stream.csv', 'w+') 
wrt.write(orders)
wrt.close()


filepath = "/home/ec2-user/trading/Trade/Positions/orders_stream.csv"
file = open(filepath,"r")
orders = file.read()
wrt = open('/home/ec2-user/trading/Trade/Positions/resent_orders' + str(today) + '.csv', 'a') 
wrt.write(orders)
wrt.close()
wrt = open('/home/ec2-user/trading/Trade/Positions/reorder_stream.csv', 'w+') 
wrt.close()
stream_order()
%time